In [34]:
import numpy as np
import os

import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms


from torch.autograd import Variable

from extractor import get_features
from extractor
os.listdir()

['.ipynb_checkpoints',
 'data',
 'RNN + loader + basic training.ipynb',
 'extractor.py',
 '__pycache__',
 'conv features extract + lstm.ipynb',
 'nohup.out',
 'object_detection',
 'Obj_detector']

In [35]:
hidden_size = 128
input_size = 512
num_layers = 2
batch_size = 2
num_epochs = 100
learning_rate = 0.01
num_classes = 3
sequence_length = 157
label_str_to_int = {'ApplyEyeMakeup': 0, 'Archery': 1, 'ApplyLipstick': 2}

In [37]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

rnn = RNN(input_size=512, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes).cuda()
rnn


RNN (
  (lstm): LSTM(512, 128, num_layers=2, batch_first=True)
  (fc): Linear (128 -> 3)
)

In [44]:
print ("GPU is available: ", torch.cuda.is_available())

GPU is available:  True


In [42]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [43]:
# for epoch in range(num_epochs):
#     for i, (labels, images) in enumerate(get_features(in_dir='data/data_subset/', batch_size=10)):
#         print (images.size())
#         labels = torch.LongTensor([label_str_to_int[i] for i in labels]).cuda()
        
# #         print (type(images), type(labels))
# #         print (images)
# #         print(labels)

#         images = Variable(images.view(-1, sequence_length, input_size).cuda())
#         labels = Variable(labels.view(-1).cuda())

#         # Forward + Backward + Optimize
#         optimizer.zero_grad()
#         outputs = rnn(images)

#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         if (i+1) % 1 == 0:
#             print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, 100000//batch_size, loss.data[0]))


Loading movie with category Archery and 153 frames
Loading movie with category Archery and 68 frames
Loading movie with category ApplyEyeMakeup and 209 frames
Loading movie with category Archery and 126 frames
Loading movie with category ApplyEyeMakeup and 159 frames
Loading movie with category ApplyEyeMakeup and 153 frames
Loading movie with category ApplyEyeMakeup and 217 frames
Loading movie with category ApplyEyeMakeup and 115 frames
Loading movie with category Archery and 142 frames
Loading movie with category ApplyEyeMakeup and 195 frames
Loading movie with category ApplyEyeMakeup and 211 frames


torch.Size([10, 157, 512])
Epoch [1/2], Step [1/50000], Loss: 1.1489


Loading movie with category Archery and 186 frames
Loading movie with category ApplyLipstick and 121 frames
Loading movie with category Archery and 65 frames
Loading movie with category ApplyLipstick and 99 frames
Loading movie with category Archery and 169 frames
Loading movie with category ApplyLipstick and 145 frames
Loading movie with category Archery and 126 frames
Loading movie with category Archery and 265 frames
Loading movie with category ApplyEyeMakeup and 103 frames
Loading movie with category Archery and 200 frames


torch.Size([10, 157, 512])
Epoch [1/2], Step [2/50000], Loss: 1.2010


Loading movie with category ApplyLipstick and 194 frames
Loading movie with category Archery and 195 frames
Loading movie with category ApplyLipstick and 126 frames
Loading movie with category ApplyEyeMakeup and 159 frames
Loading movie with category ApplyEyeMakeup and 136 frames
Loading movie with category ApplyLipstick and 263 frames
Loading movie with category ApplyEyeMakeup and 233 frames
Loading movie with category ApplyLipstick and 214 frames
Loading movie with category Archery and 73 frames
Loading movie with category ApplyEyeMakeup and 236 frames


...

torch.Size([10, 157, 512])
Epoch [2/2], Step [34/50000], Loss: 0.2042


Loading movie with category ApplyLipstick and 174 frames
Loading movie with category ApplyLipstick and 311 frames
Loading movie with category ApplyEyeMakeup and 181 frames
Loading movie with category ApplyEyeMakeup and 209 frames
Loading movie with category ApplyEyeMakeup and 210 frames
Loading movie with category Archery and 149 frames
Loading movie with category ApplyLipstick and 263 frames
Loading movie with category Archery and 355 frames
Loading movie with category ApplyEyeMakeup and 233 frames
Loading movie with category ApplyEyeMakeup and 126 frames


torch.Size([10, 157, 512])
Epoch [2/2], Step [35/50000], Loss: 0.5746


Loading movie with category ApplyEyeMakeup and 126 frames
Loading movie with category ApplyEyeMakeup and 106 frames
Loading movie with category Archery and 92 frames
Loading movie with category Archery and 148 frames
Loading movie with category ApplyLipstick and 145 frames
Loading movie with category ApplyLipstick and 121 frames
Loading movie with category Archery and 131 frames


KeyboardInterrupt: 

In [45]:
def exp_lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=7):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.1**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer


In [50]:
def check_size(address, sequence_size=50):
    x = np.load(address)
    print (x.shape)
    if x.shape[0] < sequence_size:
        return np.concatenate((x, np.zeros((sequence_size - x.shape[0], x.shape[1]))), axis=0)
    return x

# Data Loader (Input Pipeline)
def get_loader(path=''):
    """
    Function reads .npy files from path.
    Returns:
        dataloader, data classes (list), size of input object [n_sequence, n_features], lenght_of_dataset
    """
    inputs = []
    targets = []
    data_classes = os.listdir(path)
    label_int = 0
    for folder in data_classes:
        current_dir = path + '/' + folder + '/'
        
        temp = [torch.Tensor(np.load(current_dir +  f)) for f in os.listdir(current_dir)] 
                         # Transform to torch tensors
        
        targets += ([torch.LongTensor([label_int])] * len(temp))
        inputs += temp
        
        label_int += 1
        
    tensor_x = torch.stack(inputs)
    tensor_y = torch.stack(targets)
    my_dataset = torch.utils.data.TensorDataset(tensor_x, tensor_y)  # Create your datset
    my_dataloader = torch.utils.data.DataLoader(my_dataset, batch_size=batch_size, shuffle=True)  # Create your dataloader
    
    
    return (my_dataloader, data_classes)

# dset_loaders = {x: get_loader('Features/' + x)[0] for x in ['train', 'val']}

In [ ]:
def train_model(model, criterion, optimizer, lr_scheduler, path, num_epochs=200, model_name='lstm'):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                optimizer = lr_scheduler(optimizer, epoch)
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dset_loaders[phase]:
                # get the inputs
                
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), \
                        Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects / dset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model)
        # saving weights
        torch.save(model, "./trained_weights/" + model_name + '_' + str(epoch) + ".pt")

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return best_model


In [ ]:
# # Test set is not implemented yet

# # Test the Model
# correct = 0
# total = 0
# for images, labels in test_loader:
#     images = Variable(images.view(-1, sequence_length, input_size))
#     outputs = rnn(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum()

# print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

# # Save the Model
# # torch.save(rnn.state_dict(), 'rnn.pkl')
# # torch.save(rnn, 'rnn.pt')

# Generate dummy data set

In [ ]:
import numpy as np

In [ ]:
n_feat = 100
n_seq = 50 # fixed for now
n_video = 10

In [ ]:
for i in range(4,5):
    tmp_data = np.random.random((n_seq+1, n_feat))
    np.save('data/features/Archery/v' + str(i) + '.npy', tmp_data)